## The simpliest usage example of py_boost

### Installation (if needed)

**Note**: replace cupy-cuda110 with your cuda version!!!

In [1]:
# !pip install cupy-cuda110

In [1]:
# !pip uninstall py-boost
# !cd ..; bash build_package.sh
# !pip install ../dist/Py_Boost-0.1.8-py3-none-any.whl

### Imports

In [2]:
import os
# Optional: set the device to run
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

os.makedirs('../data', exist_ok=True)

import joblib
import numpy as np
from sklearn.datasets import make_regression

# simple case - just one class is used
from py_boost import GradientBoosting, CrossValidation

### Generation of dummy regression data

In [3]:
%%time
X, y = make_regression(150000, 100, n_targets=1, random_state=42)
X_test, y_test = X[:50000], y[:50000]
X, y = X[-50000:], y[-50000:]

CPU times: user 2.05 s, sys: 1.36 s, total: 3.41 s
Wall time: 792 ms


### Training a GBDT model

The only argument required here is a loss function. It, together with the input target shape, determines the task type. The loss function can be passed as a Loss instance or using a string alias:

* ***'mse'*** for the regression/multitask regression
* ***'msle'*** for the regression/multitask regression
* ***'bce'*** for the binary/multilabel classification
* ***'crossentropy'*** for the multiclassification

Training is simply done by calling the .fit metod. Possible argumentsare the following:

* ***'X'*** 
* ***'y'*** 
* ***'sample_weight'*** 
* ***'eval_sets'***  
A validation set is passed as a list of dicts with possible keys ['X', 'y', 'sample_weight']. Note: if multiple valid sets are passed, the best model is selected using the last one.

#### The example below illustrates how to train a simple regression task.

In [4]:
%%time
model = GradientBoosting('mse', ntrees=1000, verbose=100)

model.fit(X, y, eval_sets=[{'X': X_test, 'y': y_test},])

[19:46:58] Stdout logging level is INFO.
[19:46:58] GDBT train starts. Max iter 1000, early stopping rounds 100
[19:46:58] Iter 0; Sample 0, rmse = 175.08540219165158; 
[19:46:59] Iter 100; Sample 0, rmse = 34.70489960538339; 
[19:47:00] Iter 200; Sample 0, rmse = 18.875264744140104; 
[19:47:00] Iter 300; Sample 0, rmse = 15.979051235180613; 
[19:47:01] Iter 400; Sample 0, rmse = 15.373399030728404; 
[19:47:02] Iter 500; Sample 0, rmse = 15.218665059712949; 
[19:47:03] Iter 600; Sample 0, rmse = 15.164454086341598; 
[19:47:04] Iter 700; Sample 0, rmse = 15.094543783594625; 
[19:47:05] Iter 800; Sample 0, rmse = 15.029503808203762; 
[19:47:06] Iter 900; Sample 0, rmse = 14.966281468540894; 
[19:47:06] Iter 999; Sample 0, rmse = 14.916428475784219; 
CPU times: user 11.8 s, sys: 808 ms, total: 12.6 s
Wall time: 11.3 s


Here we will make prediction from different stages and will try to cluster paths

In [5]:
%%time
iters_to_pred = np.arange(45).cumsum()

test_staged_pred = model.predict_staged(X_test, iterations=iters_to_pred)
# shape - (iterations, objects, output_dim)
test_staged_pred.shape

CPU times: user 901 ms, sys: 330 ms, total: 1.23 s
Wall time: 1.25 s


(45, 50000, 1)

Calculate errors

In [6]:
error_paths = ((y_test[np.newaxis, :] - test_staged_pred[..., 0]) ** 2).T
error_paths[0]

array([7.76091618e+03, 7.91277819e+03, 6.88692028e+03, 6.62394368e+03,
       5.07055804e+03, 2.98405530e+03, 1.90369521e+03, 1.42643322e+03,
       1.02466640e+03, 5.42772313e+02, 2.73758818e+02, 1.03262352e+02,
       7.15190962e+00, 5.33730708e+00, 2.04352367e+01, 1.29329528e+01,
       9.64893900e+00, 1.40341652e+01, 9.99071383e+00, 1.22499233e+01,
       1.40068547e+01, 1.84563536e+01, 1.60256184e+01, 1.60131597e+01,
       1.84176975e+01, 1.70987199e+01, 1.78987743e+01, 1.90123040e+01,
       2.24470844e+01, 2.29833427e+01, 2.36875508e+01, 2.51050573e+01,
       2.08873075e+01, 1.90737636e+01, 1.98194222e+01, 1.88469314e+01,
       1.73786142e+01, 2.01084945e+01, 2.08516872e+01, 2.05898315e+01,
       2.08363611e+01, 2.09833754e+01, 1.89431062e+01, 1.98025111e+01,
       2.25278366e+01])

Create cluster tree

In [11]:
from py_boost.gpu.cluster_tree import ClusterCandidates

clustering = ClusterCandidates(depth_range=list(range(1, 7)), min_data_in_leaf=100)
clustering.fit(X_test, error_paths)

In [12]:
cluster_candidates = clustering.predict_leaves(X_test)
# shape - (n_candidates, n_objects, 1)
cluster_candidates.shape

(6, 50000, 1)

In [13]:
cluster_candidates[..., 0].T

array([[0, 2, 3, 4, 6, 9],
       [0, 2, 3, 4, 6, 9],
       [0, 2, 3, 4, 6, 9],
       ...,
       [0, 2, 2, 3, 5, 8],
       [0, 2, 2, 3, 4, 6],
       [0, 2, 3, 4, 6, 9]], dtype=uint32)

In [14]:
cluster_candidates[..., 0].T.max(axis=0)

array([ 1,  2,  3,  5,  7, 10], dtype=uint32)